In [1]:
## Get dependencies ##

import numpy as np
import string
import math
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn
from UnFaIR import *
import scipy as sp
import pickle
import time

In [9]:
## Get Law Dome

Law_Dome = pd.read_csv('./Conc_fit_data/law2006.txt',index_col=0,skiprows=183,nrows=2188-184,delim_whitespace=True,usecols=[0,3,5,8],names=['Year','CH4','CO2','N2O'])

# Get NOAA atmospheric observations

obsv_concs = pd.read_csv('./Conc_fit_data/NOAA_Conc_data.csv',skiprows=4,header=None,index_col=0).dropna().iloc[:,:3].values
obsv_concs = pd.DataFrame(data=obsv_concs,index = np.arange(1979,2018),columns=['CO2','CH4','N2O'])

# Get CMIP6 concs (some harmonization of Law Dome and atmospheric observations:

CMIP6_concs = pd.read_csv('./CMIP_input_ems/Supplementary_Table_UoM_GHGConcentrations-1-1-0_annualmeans_v23March2017.csv',skiprows=21,index_col=0)

## Get the other temp datasets:

NOAA_T = pd.read_csv('./Conc_fit_data/1880-2019_NOAA_1.csv',skiprows=4,index_col=0)
GISS_T = pd.read_csv('./Conc_fit_data/GLB.Ts+dSST.csv',skiprows=1,index_col=0,usecols=['Year','J-D'])
BERKELEY_T = pd.read_csv('./Conc_fit_data/Berkeley_Earth_Land_and_Ocean_complete_csv_sea_ice_air.csv',index_col=0)
CW_T = pd.read_csv('./Conc_fit_data/had4_krig_annual_v2_0_0.txt',index_col=0,delim_whitespace=True,names=['Year','Anom'],usecols=[0,1])
HadCRUT_ensemble = pd.read_csv('./Conc_fit_data/HadCRUT4_annual_obsv/HadCRUT.4.6.0.0.annual_ns_avg.1.txt',index_col=0,delim_whitespace=True,usecols=[0,1],names=['Year',1])
for i in np.arange(2,101):
    HadCRUT_ensemble[i] = pd.read_csv('./Conc_fit_data/HadCRUT4_annual_obsv/HadCRUT.4.6.0.0.annual_ns_avg.'+str(i)+'.txt',index_col=0,delim_whitespace=True,usecols=[0,1],names=['Year',i])

baseline_start = 1880
baseline_end = 1901
    
BERKELEY_T = BERKELEY_T.rolling(12).mean().iloc[11::12,1]
CW_T = CW_T['Anom']
GISS_T = GISS_T['J-D'].iloc[:-1].apply(pd.to_numeric)
NOAA_T = NOAA_T['Value']

TEMP_DATASETS = pd.DataFrame([NOAA_T,GISS_T,BERKELEY_T,CW_T,HadCRUT_ensemble.mean(axis=1)],index=['NOAA','GISS','BERKELEY','CW','HadCRUT']).transpose()
HC_UNCERT = (HadCRUT_ensemble.transpose() - HadCRUT_ensemble.mean(axis=1)).transpose()
#HC_UNCERT.plot(legend=None)

#TEMP_DATASETS.plot()

TEMP_DATASETS -= TEMP_DATASETS.loc[1880:1920].mean()
TEMP_DATASETS['Mean'] = TEMP_DATASETS.mean(axis=1)

import statsmodels.api as sm

forcings = pd.read_csv('./Conc_fit_data/Annualforcings_Mar2014_GHGrevised.txt',skiprows=3,sep='\t',index_col=0)
empty_ems = pd.DataFrame(index=forcings.index,columns=['CO2','CH4','N2O']).fillna(0.)
T_ant = UnFaIR(emissions_in=empty_ems,F_ext=forcings.Anthrototal.values)['T']
T_nat = UnFaIR(emissions_in=empty_ems,F_ext=(forcings.Total.values - forcings.Anthrototal.values))['T']

X = pd.DataFrame([T_ant.Total,T_nat.Total],index=['ANT','NAT']).transpose()
X = sm.add_constant(X)
T_responses = pd.DataFrame(index=X.index)

y = TEMP_DATASETS.Mean

ols_fit = sm.OLS(y.loc[X.index].dropna(),X.loc[y.index].dropna()).fit()
    
T_responses = (ols_fit.params[['ANT','NAT']] * X[['ANT','NAT']]).sum(axis=1)
    
## And we now have an uncertainty in our temperature response!

T_responses -= T_responses.loc[1765]

/home/nleach/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [3]:
## Get my best estimate emission series generated by the emission script.

emissions = pd.read_csv('./Best_estimate_emissions_for_UnFaIR.csv',index_col=0)

In [4]:
## Get some RCP emisisons / concentrations / forcings

RCP = '85'
RCP_E = pd.read_csv('./RCP_data/RCP'+RCP+'_EMISSIONS.csv',skiprows=36,index_col=0)
RCP_C = pd.read_csv('./RCP_data/RCP'+RCP+'_MIDYEAR_CONCENTRATIONS.csv',skiprows=37,index_col=0)
RCP_forc = pd.read_csv('./Conc_fit_data/RCP'+RCP+'_MIDYEAR_RADFORCING.csv',skiprows=58,index_col=0)
otherforc = RCP_forc.TOTAL_INCLVOLCANIC_RF.loc[:2012].values - RCP_forc.CO2CH4N2O_RF.loc[:2012].values

In [55]:
GCP = pd.read_csv('./Conc_fit_data/Global_Carbon_Budget_2018v1.0.csv',index_col=0)
CO2_GCP = GCP.fillna(0).sum(axis=1)
CO2_GCP = pd.DataFrame(data= CO2_GCP.values, index = CO2_GCP.index,columns=['CO2'])

RCP85_E = pd.read_csv('./RCP_data/RCP85_EMISSIONS.csv',skiprows=36,index_col=0)
RCP_ems = pd.DataFrame(data = np.array([RCP85_E[['OtherCO2','FossilCO2']].sum(axis=1).values,RCP85_E['CH4'].values,RCP85_E['N2O'].values]).T,columns=['CO2','CH4','N2O'],index = RCP85_E.index)

CO2_1765_2016 = (RCP_ems.loc[1765:1849,'CO2']*CO2_GCP.loc[1850,'CO2']/RCP_ems.loc[1850,'CO2']).append(CO2_GCP.loc[1850:2016,'CO2'])

emissions = empty_emissions(1765,2016)

emissions['CO2'] = CO2_1765_2016.values

In [59]:
## Questions:

# Do we fit 1850: as in other gases
# Or 1979: with decent observations
# Equal weight to present day and past lstsq

gas_params = default_gas_params()

def fit_CO2(x):
    
    fit_params = gas_params.copy()
    
    fit_params.loc[['r0','rC','rT'],'CO2'] = [x[0],x[1],x[1]*4.165/0.019]
    
    model_run = fit_gas_cycles(emissions,T_responses.loc[1765:2016],fit_params)['CO2']
    
    return np.sum((model_run.loc[1979:2016].values.flatten() - obsv_concs.loc[:2016,'CO2'].values.flatten())**2 / obsv_concs.loc[:2016,'CO2'].values.flatten()**2) + \
           (2016-1979) * (model_run.loc[2016] - obsv_concs.loc[2016,'CO2'])**2 / obsv_concs.loc[2016,'CO2']**2

In [60]:
sp.optimize.minimize(fit_CO2,[32.4,0.019],method='Nelder-Mead')

## 32.2 , 1.11

 final_simplex: (array([[2.94710200e+01, 1.79841927e-02],
       [2.94711175e+01, 1.79839558e-02],
       [2.94710000e+01, 1.79842118e-02]]), array([0.00024867, 0.00024867, 0.00024867]))
           fun: 0.0002486729568945599
       message: 'Optimization terminated successfully.'
          nfev: 89
           nit: 47
        status: 0
       success: True
             x: array([2.94710200e+01, 1.79841927e-02])